<a href="https://colab.research.google.com/github/urmzd/rabbit-classifier/blob/feature%2Fwandb-set-up/src/rc_e1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch_lightning
!pip install wandb

In [2]:
!wandb login

wandb: Currently logged in as: csci-4155-rabbit-classifier (use `wandb login --relogin` to force relogin)


In [3]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer

wandb_logger = WandbLogger(project="experiment-one")